In [1]:
# https://docs.ray.io/en/latest/tune/tutorials/tune-xgboost.html
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

from ray import tune


def train_breast_cancer(config):
 # Load dataset
 data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
 # Split into train and test set
 train_x, test_x, train_y, test_y = train_test_split(
     data, labels, test_size=0.25)
 # Build input matrices for XGBoost
 train_set = xgb.DMatrix(train_x, label=train_y)
 test_set = xgb.DMatrix(test_x, label=test_y)
 # Train the classifier
 results = {}
 xgb.train(
     config,
     train_set,
     evals=[(test_set, "eval")],
     evals_result=results,
     verbose_eval=False)
 # Return prediction accuracy
 accuracy = 1. - results["eval"]["error"][-1]
 tune.report(mean_accuracy=accuracy, done=True)


if __name__ == "__main__":
 config = {
     "objective": "binary:logistic",
     "eval_metric": ["logloss", "error"],
     "max_depth": tune.randint(1, 9),
     "min_child_weight": tune.choice([1, 2, 3]),
     "subsample": tune.uniform(0.5, 1.0),
     "eta": tune.loguniform(1e-4, 1e-1)
 }
 analysis = tune.run(
     train_breast_cancer,
     resources_per_trial={"cpu": 1},
     config=config,
     num_samples=10)

2021-10-30 17:48:48,390	WARNING function_runner.py:558 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_7d6e8_00000,RUNNING,,0.000163663,7,3,0.830452
train_breast_cancer_7d6e8_00001,PENDING,,0.0025668,5,3,0.646217
train_breast_cancer_7d6e8_00002,PENDING,,0.0876428,1,2,0.715516
train_breast_cancer_7d6e8_00003,PENDING,,0.000821323,2,3,0.648295
train_breast_cancer_7d6e8_00004,PENDING,,0.0498135,2,2,0.99377
train_breast_cancer_7d6e8_00005,PENDING,,0.0154784,5,1,0.640863
train_breast_cancer_7d6e8_00006,PENDING,,0.000448304,7,3,0.967807
train_breast_cancer_7d6e8_00007,PENDING,,0.000179152,1,1,0.877501
train_breast_cancer_7d6e8_00008,PENDING,,0.000197905,4,1,0.834753
train_breast_cancer_7d6e8_00009,PENDING,,0.0575993,8,1,0.646297


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_7d6e8_00000,RUNNING,,0.000163663,7,3,0.830452
train_breast_cancer_7d6e8_00001,RUNNING,,0.0025668,5,3,0.646217
train_breast_cancer_7d6e8_00002,RUNNING,,0.0876428,1,2,0.715516
train_breast_cancer_7d6e8_00003,RUNNING,,0.000821323,2,3,0.648295
train_breast_cancer_7d6e8_00004,RUNNING,,0.0498135,2,2,0.99377
train_breast_cancer_7d6e8_00005,RUNNING,,0.0154784,5,1,0.640863
train_breast_cancer_7d6e8_00006,RUNNING,,0.000448304,7,3,0.967807
train_breast_cancer_7d6e8_00007,RUNNING,,0.000179152,1,1,0.877501
train_breast_cancer_7d6e8_00008,PENDING,,0.000197905,4,1,0.834753
train_breast_cancer_7d6e8_00009,PENDING,,0.0575993,8,1,0.646297


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_7d6e8_00000,RUNNING,,0.000163663,7,3,0.830452
train_breast_cancer_7d6e8_00001,RUNNING,,0.0025668,5,3,0.646217
train_breast_cancer_7d6e8_00002,RUNNING,,0.0876428,1,2,0.715516
train_breast_cancer_7d6e8_00003,RUNNING,,0.000821323,2,3,0.648295
train_breast_cancer_7d6e8_00004,RUNNING,,0.0498135,2,2,0.99377
train_breast_cancer_7d6e8_00005,RUNNING,,0.0154784,5,1,0.640863
train_breast_cancer_7d6e8_00006,RUNNING,,0.000448304,7,3,0.967807
train_breast_cancer_7d6e8_00007,RUNNING,,0.000179152,1,1,0.877501
train_breast_cancer_7d6e8_00008,PENDING,,0.000197905,4,1,0.834753
train_breast_cancer_7d6e8_00009,PENDING,,0.0575993,8,1,0.646297


Result for train_breast_cancer_7d6e8_00007:
  date: 2021-10-30_17-49-01
  done: true
  experiment_id: 8a5f6558cc5145b7a3dd3116975e1679
  hostname: tobi-yoga
  iterations_since_restore: 1
  mean_accuracy: 0.895105
  node_ip: 10.80.1.25
  pid: 1291529
  time_since_restore: 10.413788795471191
  time_this_iter_s: 10.413788795471191
  time_total_s: 10.413788795471191
  timestamp: 1635605341
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7d6e8_00007
  
Result for train_breast_cancer_7d6e8_00002:
  date: 2021-10-30_17-49-02
  done: true
  experiment_id: 8830569224a0452fb33d67a2c551d7ef
  hostname: tobi-yoga
  iterations_since_restore: 1
  mean_accuracy: 0.895105
  node_ip: 10.80.1.25
  pid: 1291534
  time_since_restore: 11.241080284118652
  time_this_iter_s: 11.241080284118652
  time_total_s: 11.241080284118652
  timestamp: 1635605342
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7d6e8_00002
  
Result for train_breast_cancer_7d6e8_00003:
  date: 2021-10-30

...

Result for train_breast_cancer_7d6e8_00006:
  date: 2021-10-30_17-49-07
  done: true
  experiment_id: 9c5ff5282dcf469cac0f27be5a6ca7a2
  hostname: tobi-yoga
  iterations_since_restore: 1
  mean_accuracy: 0.958042
  node_ip: 10.80.1.25
  pid: 1291528
  time_since_restore: 16.382075548171997
  time_this_iter_s: 16.382075548171997
  time_total_s: 16.382075548171997
  timestamp: 1635605347
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7d6e8_00006
  
Result for train_breast_cancer_7d6e8_00000:
  date: 2021-10-30_17-49-07
  done: true
  experiment_id: d3cb0acc973f4abda8ca4d2c8caa18a3
  hostname: tobi-yoga
  iterations_since_restore: 1
  mean_accuracy: 0.902098
  node_ip: 10.80.1.25
  pid: 1291535
  time_since_restore: 16.323427438735962
  time_this_iter_s: 16.323427438735962
  time_total_s: 16.323427438735962
  timestamp: 1635605347
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7d6e8_00000
  
Result for train_breast_cancer_7d6e8_00001:
  date: 2021-10-30

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_7d6e8_00008,RUNNING,,0.000197905,4,1,0.834753,,,
train_breast_cancer_7d6e8_00009,RUNNING,,0.0575993,8,1,0.646297,,,
train_breast_cancer_7d6e8_00000,TERMINATED,,0.000163663,7,3,0.830452,0.902098,1,16.3234
train_breast_cancer_7d6e8_00001,TERMINATED,,0.0025668,5,3,0.646217,0.951049,1,16.3052
train_breast_cancer_7d6e8_00002,TERMINATED,,0.0876428,1,2,0.715516,0.895105,1,11.2411
train_breast_cancer_7d6e8_00003,TERMINATED,,0.000821323,2,3,0.648295,0.937063,1,13.2241
train_breast_cancer_7d6e8_00004,TERMINATED,,0.0498135,2,2,0.99377,0.895105,1,13.7572
train_breast_cancer_7d6e8_00005,TERMINATED,,0.0154784,5,1,0.640863,0.93007,1,16.9991
train_breast_cancer_7d6e8_00006,TERMINATED,,0.000448304,7,3,0.967807,0.958042,1,16.3821
train_breast_cancer_7d6e8_00007,TERMINATED,,0.000179152,1,1,0.877501,0.895105,1,10.4138


Result for train_breast_cancer_7d6e8_00008:
  date: 2021-10-30_17-49-12
  done: true
  experiment_id: e243ef44ad574a6ea690f8b513fdfd2d
  hostname: tobi-yoga
  iterations_since_restore: 1
  mean_accuracy: 0.937063
  node_ip: 10.80.1.25
  pid: 1292039
  time_since_restore: 4.438787937164307
  time_this_iter_s: 4.438787937164307
  time_total_s: 4.438787937164307
  timestamp: 1635605352
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7d6e8_00008
  
Result for train_breast_cancer_7d6e8_00009:
  date: 2021-10-30_17-49-12
  done: true
  experiment_id: 31d4270e930c4c4487a450ced40f82de
  hostname: tobi-yoga
  iterations_since_restore: 1
  mean_accuracy: 0.944056
  node_ip: 10.80.1.25
  pid: 1292055
  time_since_restore: 3.843602418899536
  time_this_iter_s: 3.843602418899536
  time_total_s: 3.843602418899536
  timestamp: 1635605352
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7d6e8_00009
  


Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_7d6e8_00000,TERMINATED,,0.000163663,7,3,0.830452,0.902098,1,16.3234
train_breast_cancer_7d6e8_00001,TERMINATED,,0.0025668,5,3,0.646217,0.951049,1,16.3052
train_breast_cancer_7d6e8_00002,TERMINATED,,0.0876428,1,2,0.715516,0.895105,1,11.2411
train_breast_cancer_7d6e8_00003,TERMINATED,,0.000821323,2,3,0.648295,0.937063,1,13.2241
train_breast_cancer_7d6e8_00004,TERMINATED,,0.0498135,2,2,0.99377,0.895105,1,13.7572
train_breast_cancer_7d6e8_00005,TERMINATED,,0.0154784,5,1,0.640863,0.93007,1,16.9991
train_breast_cancer_7d6e8_00006,TERMINATED,,0.000448304,7,3,0.967807,0.958042,1,16.3821
train_breast_cancer_7d6e8_00007,TERMINATED,,0.000179152,1,1,0.877501,0.895105,1,10.4138
train_breast_cancer_7d6e8_00008,TERMINATED,,0.000197905,4,1,0.834753,0.937063,1,4.43879
train_breast_cancer_7d6e8_00009,TERMINATED,,0.0575993,8,1,0.646297,0.944056,1,3.8436


2021-10-30 17:49:12,319	INFO tune.py:617 -- Total run time: 23.93 seconds (23.38 seconds for the tuning loop).


In [6]:
# Get a dataframe for the last reported results of all of the trials
df = analysis.results_df
#print(df)

# Get a dict mapping {trial logdir -> dataframes} for all trials in the experiment.
all_dataframes = analysis.trial_dataframes

# Get a list of trials
trials = analysis.trials

In [10]:
# Get a dataframe for the max accuracy seen for each trial
analysis.dataframe(metric="mean_accuracy", mode="max")

,mean_accuracy,done,time_this_iter_s,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,...,timesteps_since_restore,iterations_since_restore,trial_id,config/eta,config/eval_metric,config/max_depth,config/min_child_weight,config/objective,config/subsample,logdir
0,0.902098,True,16.323427,NaN,NaN,1,d3cb0acc973f4abda8ca4d2c8caa18a3,2021-10-30_17-49-07,1635605347,16.323427,...,0,1,7d6e8_00000,0.000164,"[logloss, error]",7,3,binary:logistic,0.830452,/home/weiss/ray_results/train_breast_cancer_20...
1,0.951049,True,16.305232,NaN,NaN,1,6f48e90517bb46329835b1d938f72bcf,2021-10-30_17-49-07,1635605347,16.305232,...,0,1,7d6e8_00001,0.002567,"[logloss, error]",5,3,binary:logistic,0.646217,/home/weiss/ray_results/train_breast_cancer_20...
2,0.895105,True,11.241080,NaN,NaN,1,8830569224a0452fb33d67a2c551d7ef,2021-10-30_17-49-02,1635605342,11.241080,...,0,1,7d6e8_00002,0.087643,"[logloss, error]",1,2,binary:logistic,0.715516,/home/weiss/ray_results/train_breast_cancer_20...
3,0.937063,True,13.224133,NaN,NaN,1,5855ed8b5f674891b400aa8773aacfde,2021-10-30_17-49-04,1635605344,13.224133,...,0,1,7d6e8_00003,0.000821,"[logloss, error]",2,3,binary:logistic,0.648295,/home/weiss/ray_results/train_breast_cancer_20...
4,0.895105,True,13.757210,NaN,NaN,1,4648bb2d4d0740908f1de50946d86317,2021-10-30_17-49-05,1635605345,13.757210,...,0,1,7d6e8_00004,0.049813,"[logloss, error]",2,2,binary:logistic,0.993770,/home/weiss/ray_results/train_breast_cancer_20...
5,0.930070,True,16.999072,NaN,NaN,1,dc8cff9925f742629d8386c53eeab966,2021-10-30_17-49-08,1635605348,16.999072,...,0,1,7d6e8_00005,0.015478,"[logloss, error]",5,1,binary:logistic,0.640863,/home/weiss/ray_results/train_breast_cancer_20...
6,0.958042,True,16.382076,NaN,NaN,1,9c5ff5282dcf469cac0f27be5a6ca7a2,2021-10-30_17-49-07,1635605347,16.382076,...,0,1,7d6e8_00006,0.000448,"[logloss, error]",7,3,binary:logistic,0.967807,/home/weiss/ray_results/train_breast_cancer_20...
7,0.895105,True,10.413789,NaN,NaN,1,8a5f6558cc5145b7a3dd3116975e1679,2021-10-30_17-49-01,1635605341,10.413789,...,0,1,7d6e8_00007,0.000179,"[logloss, error]",1,1,binary:logistic,0.877501,/home/weiss/ray_results/train_breast_cancer_20...
8,0.937063,True,4.438788,NaN,NaN,1,e243ef44ad574a6ea690f8b513fdfd2d,2021-10-30_17-49-12,1635605352,4.438788,...,0,1,7d6e8_00008,0.000198,"[logloss, error]",4,1,binary:logistic,0.834753,/home/weiss/ray_results/train_breast_cancer_20...
9,0.944056,True,3.843602,NaN,NaN,1,31d4270e930c4c4487a450ced40f82de,2021-10-30_17-49-12,1635605352,3.843602,...,0,1,7d6e8_00009,0.057599,"[logloss, error]",8,1,binary:logistic,0.646297,/home/weiss/ray_results/train_breast_cancer_20...


In [14]:
analysis.get_best_config(metric="mean_accuracy", mode="max")  # Get best trial's hyperparameters

{'objective': 'binary:logistic',
 'eval_metric': ['logloss', 'error'],
 'max_depth': 7,
 'min_child_weight': 3,
 'subsample': 0.9678065627927879,
 'eta': 0.0004483038172557843}